## 30s k-fold

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, Dropout
from keras.layers import Activation, BatchNormalization
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from toolkit import plot_confusion_matrix
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from toolkit import to_3_classes
np.set_printoptions(suppress=True)

trainD = np.load("/home/hsiehch/30s/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/validation_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testD = np.load("/home/hsiehch/30s/test_data.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainD = np.append(trainD, validationD, axis=0)
trainL = np.append(trainL, validationL, axis=0)
trainD = np.append(trainD, testD, axis=0)
trainL = np.append(trainL, testL, axis=0)

trainL = to_3_classes.to_3_classes(trainL)

trainData = trainD.reshape((trainD.shape[0], trainD.shape[1], 1))
trainLabel = np_utils.to_categorical(trainL, 3)

print('Train Data:', trainData.shape)
print('Train Label: ', trainLabel.shape)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 3)


In [2]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5, shuffle=False)
print(kf)

for train_index, test_index in kf.split(trainD, trainL):
    print('trian:', train_index, 'len', len(train_index), 'test:', test_index, 'len', len(test_index))

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
trian: [  181   182   183 ... 10148 10149 10150] len 8120 test: [   0    1    2 ... 3287 3288 3289] len 2031
trian: [    0     1     2 ... 10148 10149 10150] len 8120 test: [ 181  182  183 ... 4479 4480 4481] len 2031
trian: [    0     1     2 ... 10148 10149 10150] len 8120 test: [ 362  363  364 ... 6511 6512 6513] len 2031
trian: [    0     1     2 ... 10148 10149 10150] len 8121 test: [5169 5170 5171 ... 8957 8958 8959] len 2030
trian: [   0    1    2 ... 8957 8958 8959] len 8123 test: [ 7200  7201  7202 ... 10148 10149 10150] len 2028


In [ ]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import itertools

def create_model():
    model = Sequential()
    model.add(Conv1D(filters = 32, kernel_size = 7, input_shape = (trainData.shape[1], 1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 32, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(3, activation = "softmax"))

    return model

def calculate_avg_F1(scores):
    tmp = [0,0,0]
    for i in scores:
        tmp += i
    tmp /= 5
    return tmp

def run_model(batch_S, learning_rate):
    
    Scores = []
    for train_index, test_index in kf.split(trainD, trainL):

        model = create_model()
        adam = Adam(lr = learning_rate)
        model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])
        early_stop = EarlyStopping(patience=20)
        cw = class_weight.compute_sample_weight('balanced', [0,1,2], trainL[train_index])
        cw = {0: cw[0],
              1: cw[1],
              2: cw[2]}
        train_history_1 = model.fit(x = trainData[train_index],
                                    y = trainLabel[train_index],
                                    epochs=150,
                                    validation_data=(trainData[test_index], trainLabel[test_index]),
                                    callbacks=[early_stop],
                                    class_weight=cw,
                                    batch_size=batch_S, 
                                    verbose=0)

        evaluation = model.evaluate(x = trainData[test_index], y = trainLabel[test_index])
        print('Loss: {:.3f}, Accuracy: {:.3f}'.format(evaluation[0], evaluation[1]))

        validation_prediction = model.predict_classes(trainData[test_index], batch_size=batch_S)
        validation_prediction = np_utils.to_categorical(validation_prediction, 3)
        score = f1_score(trainLabel[test_index], validation_prediction, average=None)
        Scores.append(score)
        del model
    
    result = calculate_avg_F1(Scores)
    F1_result.append(result)
    del Scores
    

        
F1_result = []
hyperparameter_index = []
batch_S = [30, 50, 70, 90, 120]
learning_rate = [0.00005, 0.0001, 0.0005, 0.001, 0.005]

for bat, lr in itertools.product(batch_S, learning_rate):
    hyperparameter_index.append([bat, lr])
    run_model(bat, lr)

print('Finish training!')

2031/2031 [==============================] - 1s 341us/step
Loss: 0.575, Accuracy: 0.797
2031/2031 [==============================] - 1s 333us/step
Loss: 0.475, Accuracy: 0.831
2031/2031 [==============================] - 1s 338us/step
Loss: 0.498, Accuracy: 0.805
2030/2030 [==============================] - 1s 339us/step
Loss: 0.572, Accuracy: 0.802
2028/2028 [==============================] - 1s 364us/step
Loss: 0.544, Accuracy: 0.807
2031/2031 [==============================] - 1s 346us/step
Loss: 0.490, Accuracy: 0.825
2031/2031 [==============================] - 1s 342us/step
Loss: 0.492, Accuracy: 0.844
2031/2031 [==============================] - 1s 348us/step
Loss: 0.479, Accuracy: 0.821
2030/2030 [==============================] - 1s 347us/step
Loss: 0.521, Accuracy: 0.817
2028/2028 [==============================] - 1s 361us/step
Loss: 0.587, Accuracy: 0.790
2031/2031 [==============================] - 1s 362us/step
Loss: 1.097, Accuracy: 0.587


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2031/2031 [==============================] - 1s 364us/step
Loss: 1.098, Accuracy: 0.587
2031/2031 [==============================] - 1s 360us/step
Loss: 1.097, Accuracy: 0.587
2030/2030 [==============================] - 1s 396us/step
Loss: 1.095, Accuracy: 0.587
2028/2028 [==============================] - 1s 381us/step
Loss: 1.098, Accuracy: 0.587
2031/2031 [==============================] - 1s 465us/step
Loss: 1.098, Accuracy: 0.324


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2031/2031 [==============================] - 1s 426us/step
Loss: 1.099, Accuracy: 0.089
2031/2031 [==============================] - 1s 375us/step
Loss: 1.097, Accuracy: 0.587
2030/2030 [==============================] - 1s 386us/step
Loss: 1.097, Accuracy: 0.587
2028/2028 [==============================] - 1s 397us/step
Loss: 1.101, Accuracy: 0.089
2031/2031 [==============================] - 1s 397us/step
Loss: 1.098, Accuracy: 0.324


...

2031/2031 [==============================] - 1s 505us/step
Loss: 6.658, Accuracy: 0.587
2031/2031 [==============================] - 1s 481us/step
Loss: 1.109, Accuracy: 0.089
2030/2030 [==============================] - 1s 539us/step
Loss: 1.098, Accuracy: 0.587
2028/2028 [==============================] - 1s 504us/step
Loss: 1.085, Accuracy: 0.324
2031/2031 [==============================] - 1s 513us/step
Loss: 0.541, Accuracy: 0.808
2031/2031 [==============================] - 1s 545us/step
Loss: 0.478, Accuracy: 0.835
2031/2031 [==============================] - 1s 553us/step
Loss: 0.494, Accuracy: 0.820
2030/2030 [==============================] - 1s 507us/step
Loss: 0.530, Accuracy: 0.822
2028/2028 [==============================] - 1s 538us/step
Loss: 0.607, Accuracy: 0.777
2031/2031 [==============================] - 1s 514us/step
Loss: 0.512, Accuracy: 0.819
2031/2031 [==============================] - 1s 553us/step
Loss: 0.453, Accuracy: 0.848
2031/2031 [=====================

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2031/2031 [==============================] - 1s 557us/step
Loss: 1.098, Accuracy: 0.587
2031/2031 [==============================] - 1s 579us/step
Loss: 1.099, Accuracy: 0.324
2030/2030 [==============================] - 1s 581us/step
Loss: 1.099, Accuracy: 0.089
2028/2028 [==============================] - 1s 572us/step
Loss: 1.095, Accuracy: 0.587
2031/2031 [==============================] - 1s 589us/step
Loss: 1.098, Accuracy: 0.324


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2031/2031 [==============================] - 1s 586us/step
Loss: 1.094, Accuracy: 0.587
2031/2031 [==============================] - 1s 620us/step
Loss: 1.103, Accuracy: 0.089
2030/2030 [==============================] - 1s 583us/step
Loss: 1.103, Accuracy: 0.324
2028/2028 [==============================] - 1s 587us/step
Loss: 1.102, Accuracy: 0.324
2031/2031 [==============================] - 1s 628us/step
Loss: 0.484, Accuracy: 0.810
2031/2031 [==============================] - 1s 584us/step
Loss: 0.521, Accuracy: 0.810
2031/2031 [==============================] - 1s 614us/step
Loss: 0.465, Accuracy: 0.824
2030/2030 [==============================] - 1s 606us/step
Loss: 0.510, Accuracy: 0.805
2028/2028 [==============================] - 1s 606us/step
Loss: 0.682, Accuracy: 0.731


In [ ]:
import pprint
pprint.pprint(F1_result)